In [ ]:
import os
import re
import pywintypes
import win32file
import win32con
import datetime
import piexif
import subprocess
from PIL import Image
from PIL.ExifTags import TAGS

# ------------------ CONFIG ------------------
ROOT_DIR = r'c:\Users\kuste\Desktop\project\Arhiv slik'
IMAGE_EXTS = ['.jpg', '.jpeg', '.JPG', '.JPEG', '.png', '.PNG']  # Added PNG formats
VIDEO_EXTS = ['.mp4', '.MP4']
YEAR_PATTERN = re.compile(r'\b(20[0-2][0-9]|2030)\b')  # 2000-2030
EXIFTOOL_PATH = r'C:\ProgramData\chocolatey\bin\exiftool.exe'
# --------------------------------------------

def get_folder_year(folder_name):
    """Extract year from folder name if present"""
    match = YEAR_PATTERN.search(folder_name)
    return int(match.group()) if match else None

def get_image_date(image_path):
    """Get Date Taken from image EXIF (works for JPG/PNG)"""
    try:
        with Image.open(image_path) as img:
            # PNG files store creation time in different metadata
            if image_path.lower().endswith(('.png', '.PNG')):
                try:
                    # Try to get PNG creation time from text chunks
                    for chunk in img.info:
                        if 'creation time' in chunk.lower():
                            return datetime.datetime.strptime(img.info[chunk], '%Y:%m:%d %H:%M:%S')
                except:
                    return None
            
            # Standard EXIF handling for JPG
            exif = img._getexif()
            if exif:
                for tag, value in exif.items():
                    if TAGS.get(tag) == 'DateTimeOriginal':
                        return datetime.datetime.strptime(value, '%Y:%m:%d %H:%M:%S')
    except Exception:
        return None

def get_video_date(video_path):
    """Get Media Created date using ExifTool"""
    try:
        result = subprocess.run(
            [EXIFTOOL_PATH, '-MediaCreateDate', '-d', '%Y:%m:%d %H:%M:%S', '-s', '-S', video_path],
            capture_output=True, text=True, creationflags=subprocess.CREATE_NO_WINDOW
        )
        if result.returncode == 0 and result.stdout.strip():
            return datetime.datetime.strptime(result.stdout.strip(), '%Y:%m:%d %H:%M:%S')
    except Exception:
        return None

def set_filesystem_dates(file_path, new_date):
    """Update Created/Modified dates"""
    try:
        win_time = pywintypes.Time(new_date)
        handle = win32file.CreateFile(
            file_path,
            win32con.GENERIC_WRITE,
            win32con.FILE_SHARE_READ | win32con.FILE_SHARE_WRITE,
            None,
            win32con.OPEN_EXISTING,
            win32con.FILE_ATTRIBUTE_NORMAL,
            None
        )
        win32file.SetFileTime(handle, win_time, win_time, win_time)
        handle.Close()
        return True
    except Exception:
        return False

def update_image_metadata(image_path, new_date):
    """Update EXIF metadata for images (JPG only)"""
    try:
        if not image_path.lower().endswith(('.jpg', '.jpeg', '.JPG', '.JPEG')):
            return False  # Skip metadata update for PNGs
            
        exif_dict = piexif.load(image_path) or {"0th": {}, "Exif": {}, "GPS": {}, "1st": {}}
        date_str = new_date.strftime("%Y:%m:%d %H:%M:%S")
        exif_dict['Exif'][piexif.ExifIFD.DateTimeOriginal] = date_str
        piexif.insert(piexif.dump(exif_dict), image_path)
        return True
    except Exception:
        return False

def update_video_metadata(video_path, new_date):
    """Update all video date fields"""
    try:
        date_str = new_date.strftime('%Y:%m:%d %H:%M:%S')
        subprocess.run(
            [EXIFTOOL_PATH,
             '-CreateDate=' + date_str,
             '-MediaCreateDate=' + date_str,
             '-TrackCreateDate=' + date_str,
             '-overwrite_original',
             video_path],
            check=True,
            creationflags=subprocess.CREATE_NO_WINDOW
        )
        return True
    except Exception:
        return False

def find_closest_media_date(folder_path, is_video_file=False):
    """Find closest media date in folder matching the file type"""
    dates = []
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        if is_video_file and file.lower().endswith(tuple(VIDEO_EXTS)):
            date = get_video_date(file_path)
        elif not is_video_file and file.lower().endswith(tuple(IMAGE_EXTS)):
            date = get_image_date(file_path)
        else:
            continue
        if date:
            dates.append(date)
    return min(dates, key=lambda x: x) if dates else None

def process_file(file_path, folder_year, force_date=None, folder_path=None):
    """Process individual file with exact specified logic"""
    if file_path.lower().endswith(tuple(IMAGE_EXTS)):
        # PHOTO PROCESSING (JPG/PNG)
        file_date = get_image_date(file_path)
        
        if force_date:
            if file_path.lower().endswith(('.jpg', '.jpeg', '.JPG', '.JPEG')):
                update_image_metadata(file_path, force_date)
            set_filesystem_dates(file_path, force_date)
            return True
            
        if folder_year:
            if file_date and file_date.year == folder_year:
                set_filesystem_dates(file_path, file_date)
                return True
            else:
                fallback_date = datetime.datetime(folder_year, 6, 15)
                if file_path.lower().endswith(('.jpg', '.jpeg', '.JPG', '.JPEG')):
                    update_image_metadata(file_path, fallback_date)
                set_filesystem_dates(file_path, fallback_date)
                return True
        else:
            if file_date:
                set_filesystem_dates(file_path, file_date)
                return True
            else:
                closest_date = find_closest_media_date(folder_path, False)
                if closest_date:
                    set_filesystem_dates(file_path, closest_date)
                    return True
    
    elif file_path.lower().endswith(tuple(VIDEO_EXTS)):
        # VIDEO PROCESSING
        file_date = get_video_date(file_path)
        
        if force_date:
            update_video_metadata(file_path, force_date)
            set_filesystem_dates(file_path, force_date)
            return True
            
        if folder_year:
            if file_date and file_date.year == folder_year:
                update_video_metadata(file_path, file_date)
                set_filesystem_dates(file_path, file_date)
                return True
            else:
                fallback_date = datetime.datetime(folder_year, 6, 15)
                update_video_metadata(file_path, fallback_date)
                set_filesystem_dates(file_path, fallback_date)
                return True
        else:
            if file_date:
                update_video_metadata(file_path, file_date)
                set_filesystem_dates(file_path, file_date)
                return True
            else:
                closest_date = find_closest_media_date(folder_path, True)
                if closest_date:
                    update_video_metadata(file_path, closest_date)
                    set_filesystem_dates(file_path, closest_date)
                    return True
                else:
                    return False
    
    return False

def process_folder(folder_path):
    """Process all files in a folder with exact specified logic"""
    folder_name = os.path.basename(folder_path)
    folder_year = get_folder_year(folder_name)
    updated_count = 0

    # Check for date.txt (highest priority)
    txt_path = os.path.join(folder_path, 'date.txt')
    force_date = None
    if os.path.exists(txt_path):
        try:
            with open(txt_path, 'r', encoding='utf-8') as f:
                date_str = f.readline().strip()
                date_str = ''.join(c for c in date_str if c.isprintable())
                force_date = datetime.datetime.strptime(date_str, "%A, %d %B %Y")
                print(f"ℹ️ Using forced date from date.txt: {force_date.date()}")
        except Exception as e:
            print(f"⚠️ Error reading date.txt: {e}")

    for filename in sorted(os.listdir(folder_path)):
        file_path = os.path.join(folder_path, filename)
        
        if not (file_path.lower().endswith(tuple(IMAGE_EXTS + VIDEO_EXTS))):
            continue

        if process_file(file_path, folder_year, force_date, folder_path):
            updated_count += 1
            print(f"✔️ Updated: {filename}")
        else:
            print(f"⚠️ Could not process: {filename}")

    return updated_count

def main():
    # Verify ExifTool is available
    try:
        subprocess.run([EXIFTOOL_PATH, '-ver'], check=True, 
                      capture_output=True, creationflags=subprocess.CREATE_NO_WINDOW)
    except Exception:
        print("❌ ExifTool not found. Install via: choco install exiftool")
        return

    total_updated = 0
    for root, dirs, files in os.walk(ROOT_DIR):
        print(f"\n📂 Processing folder: {os.path.basename(root)}")
        updated = process_folder(root)
        total_updated += updated
        print(f"Updated {updated} files in this folder")
    
    print(f"\n✅ Total files updated: {total_updated}")

if __name__ == "__main__":
    main()


📂 Processing folder: Arhiv slik
Updated 0 files in this folder

📂 Processing folder: !Prema gore kranjske
✔️ Updated: IMG_20210920_111621.jpg
✔️ Updated: IMG_20210920_112058.jpg
✔️ Updated: IMG_20210921_172137.jpg
✔️ Updated: IMG_20210921_173506.jpg
✔️ Updated: IMG_20210921_175826.jpg
✔️ Updated: IMG_20210923_130216.jpg
✔️ Updated: IMG_20210923_204824-je uredil(-a)_.jpg
✔️ Updated: IMG_20210923_204824.jpg
✔️ Updated: IMG_20210923_204837-je uredil(-a)_.jpg
✔️ Updated: IMG_20210923_204837.jpg
⚠️ Could not process: VID_20210922_180333~2.mp4
✔️ Updated: received_3173426426226982.jpeg
Updated 11 files in this folder

📂 Processing folder: Arhiv
✔️ Updated: .trashed-1739827208-WallX_1349137_1080x1920.jpeg
✔️ Updated: 03f53d3f919db3b017d5b66e19dece65.jpg
✔️ Updated: 1119991.jpg
✔️ Updated: 137374_original_2738x3000.jpg
✔️ Updated: 1d14f37c8047758a46100cdaca46d794.jpg
✔️ Updated: 4k-android-i6wkk4dtdymct63b.jpg
✔️ Updated: 57166.jpg
✔️ Updated: 6283118.jpg
✔️ Updated: 7c4eee0569daa6c5ff56613ff